In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

In [8]:
df = pd.read_csv("Zomato_Restaurants_All_Consolidated-Types.csv")
df

,Unnamed: 0,Name,Aggregate Rating,Type,Price,Latitude,Longitude
0,0,Doughboys Doughnuts,4.0,Cafe Food,1,-37.815951,144.957859
1,1,Bluestones Cafe,3.0,Cafe Food,2,-37.815490,144.958588
2,2,Sakae,2.7,Japanese,2,-37.815578,144.957547
3,3,Crisp,3.8,Healthy Food,2,-37.815578,144.957509
4,4,Pho Melbourne,3.1,Vietnamese,2,-37.815073,144.958033
...,...,...,...,...,...,...,...
370,395,MoVida Bar De Tapas,4.7,European,4,-37.816703,144.969190
371,396,VB Rolls,3.4,Vietnamese,2,-37.813841,144.966583
372,397,Sushi Land,3.3,Japanese,2,-37.815165,144.971400
373,398,Final Pass,3.4,Drinks Only,1,-37.813158,144.967223


In [9]:
sensors_df = pd.read_csv("PCS_Locations.csv")
sensors_df

,sensor_id,sensor_description,sensor_name,installation_date,status,note,direction_1,direction_2,latitude,longitude,location
0,55,Elizabeth St-La Trobe St (East),Eli380_T,2018/07/19,A,NaN,North,South,-37.809889,144.961343,"(-37.80988941, 144.96134331)"
1,45,Little Collins St-Swanston St (East),Swa148_T,2017/06/29,A,NaN,North,South,-37.814141,144.966094,"(-37.81414074, 144.9660938)"
2,61,RMIT Building 14,RMIT14_T,2019/06/28,A,NaN,North,South,-37.807675,144.963091,"(-37.80767455, 144.96309115)"
3,7,Birrarung Marr,Fed_T,2014/12/17,A,NaN,East,West,-37.818629,144.971694,"(-37.81862929, 144.97169395)"
4,72,Flinders St- ACMI,ACMI_T,2020/11/30,A,NaN,East,West,-37.817263,144.968728,"(-37.81726337, 144.96872809)"
...,...,...,...,...,...,...,...,...,...,...,...
69,51,QVM-Franklin St (North),Fra118_T,2017/11/30,A,NaN,West,East,-37.808418,144.959063,"(-37.80841814, 144.95906317)"
70,10,Victoria Point,BouHbr_T,2009/04/23,A,NaN,West,East,-37.818765,144.947105,"(-37.81876473, 144.94710545)"
71,43,Monash Rd-Swanston St (West),UM2_T,2015/04/15,A,NaN,South,North,-37.798445,144.964118,"(-37.79844525, 144.96411782)"
72,73,Bourke St - Spencer St (South),Bou655_T,2020/10/02,A,NaN,West,East,-37.816957,144.954154,"(-37.81695684, 144.95415373)"


In [15]:
sensor_descriptions = ["Bourke Street Mall (North)", "Collins Place (South)", "Lonsdale St-Spring St (West)", "Flinders St - ACMI"]

reduced_sensors = sensors_df.loc[(sensors_df["sensor_description"] == "Bourke Street Mall (North)") | (sensors_df["sensor_description"] == "Collins Place (South)") |
               (sensors_df["sensor_description"] == "Lonsdale St-Spring St (West)") | (sensors_df["sensor_description"] == "Flinders St- ACMI"),:]

reduced_sensors

,sensor_id,sensor_description,sensor_name,installation_date,status,note,direction_1,direction_2,latitude,longitude,location
4,72,Flinders St- ACMI,ACMI_T,2020/11/30,A,NaN,East,West,-37.817263,144.968728,"(-37.81726337, 144.96872809)"
13,17,Collins Place (South),Col15_T,2009/03/30,A,Device is upgraded in 26/02/2020,East,West,-37.813625,144.973236,"(-37.81362542, 144.97323592)"
46,1,Bourke Street Mall (North),Bou292_T,2009/03/24,A,NaN,West,East,-37.813494,144.965153,"(-37.8134944, 144.96515324)"
51,40,Lonsdale St-Spring St (West),Spr201_T,2015/01/19,A,NaN,South,North,-37.809993,144.972276,"(-37.80999341, 144.97227588)"


In [61]:
lat = [list(reduced_sensors["latitude"])[0]-1] + list(reduced_sensors["latitude"])
lon = [list(reduced_sensors["longitude"].sort_values())[0]-1] + list(reduced_sensors["longitude"].sort_values())
loc_labels = reduced_sensors["sensor_description"]

df["Lat"] = pd.cut(df["Latitude"], lat, labels=loc_labels)
df["Lon"] = pd.cut(df["Longitude"], lon, labels=loc_labels)
df

,Unnamed: 0,Name,Aggregate Rating,Type,Price,Latitude,Longitude,Lat,Lon
0,0,Doughboys Doughnuts,4.0,Cafe Food,1,-37.815951,144.957859,Collins Place (South),Flinders St- ACMI
1,1,Bluestones Cafe,3.0,Cafe Food,2,-37.815490,144.958588,Collins Place (South),Flinders St- ACMI
2,2,Sakae,2.7,Japanese,2,-37.815578,144.957547,Collins Place (South),Flinders St- ACMI
3,3,Crisp,3.8,Healthy Food,2,-37.815578,144.957509,Collins Place (South),Flinders St- ACMI
4,4,Pho Melbourne,3.1,Vietnamese,2,-37.815073,144.958033,Collins Place (South),Flinders St- ACMI
...,...,...,...,...,...,...,...,...,...
370,395,MoVida Bar De Tapas,4.7,European,4,-37.816703,144.969190,Collins Place (South),Bourke Street Mall (North)
371,396,VB Rolls,3.4,Vietnamese,2,-37.813841,144.966583,Collins Place (South),Collins Place (South)
372,397,Sushi Land,3.3,Japanese,2,-37.815165,144.971400,Collins Place (South),Bourke Street Mall (North)
373,398,Final Pass,3.4,Drinks Only,1,-37.813158,144.967223,Lonsdale St-Spring St (West),Collins Place (South)


In [65]:
df["Lat"].value_counts(), df["Lon"].value_counts()

(Collins Place (South)           193
 Lonsdale St-Spring St (West)     71
 Bourke Street Mall (North)       14
 Flinders St- ACMI                 3
 Name: Lat, dtype: int64,
 Flinders St- ACMI               280
 Collins Place (South)            52
 Bourke Street Mall (North)       43
 Lonsdale St-Spring St (West)      0
 Name: Lon, dtype: int64)

In [72]:
df.loc[df["Lat"] == df["Lon"],:].sort_values(by = "Latitude")

,Unnamed: 0,Name,Aggregate Rating,Type,Price,Latitude,Longitude,Lat,Lon
314,338,Saving Grace,3.5,Bar Food,3,-37.817446,144.957702,Flinders St- ACMI,Flinders St- ACMI
312,336,Burger Boys Melbourne,3.8,Burger,3,-37.817435,144.957701,Flinders St- ACMI,Flinders St- ACMI
303,327,The Joint Melbourne,0.0,Cafe Food,2,-37.817299,144.957482,Flinders St- ACMI,Flinders St- ACMI
295,319,Chapter House Coffee,3.8,Cafe Food,2,-37.816415,144.967891,Collins Place (South),Collins Place (South)
210,229,Om Nom Kitchen - Adelphi Hotel,4.3,Desserts,4,-37.816210,144.968313,Collins Place (South),Collins Place (South)
146,158,Kisumé,4.5,Japanese,4,-37.816092,144.968640,Collins Place (South),Collins Place (South)
147,159,Supernormal,4.5,Asian,4,-37.816048,144.968372,Collins Place (South),Collins Place (South)
132,143,Dessous,3.4,Bar Food,3,-37.815948,144.968677,Collins Place (South),Collins Place (South)
217,237,Allegro Restaurant,3.7,European,4,-37.815929,144.967486,Collins Place (South),Collins Place (South)
215,234,Lobby Lounge - The Westin Melbourne,3.9,Australian,3,-37.815896,144.967482,Collins Place (South),Collins Place (South)


In [5]:
# Access maps with unique API key
gmaps.configure(api_key=gkey)

In [67]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Name}</dd>
<dt>Type</dt><dd>{Type}</dd>
<dt>Rating</dt><dd>{Aggregate Rating}</dd>
<dt>Price</dt><dd>{Price}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
eatery_info = [info_box_template.format(**row) for index, row in df.iterrows()]
locations = df[["Latitude", "Longitude"]]

In [69]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=eatery_info)

# Plot Heatmap
fig = gmaps.figure()

# Add marker layer ontop of heat map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# params dictionary to update each iteration
params = {
    "radius": 50,
    "types": ["cafe","bar","bakery","meal_takeaway","restaurant"],
    #"keyword": "hotel",
    "key": gkey
}
# Use the lat/lng from hotel_df to find nearby hotels
for index, row in df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and iterate through lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and convert to json
    name_address = requests.get(base_url, params=params).json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        df.loc[index, "Google Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        df.loc[index, "Google Name"] = None


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


KeyboardInterrupt: 